In [12]:
import pandas as pd
import numpy as np
import pickle

with open('WESAD 4/All_ID.pkl', 'rb') as file:
    data = pickle.load(file, encoding='latin1')

In [13]:
data

{'data': {'ACC': array([[[ 55., -32.,  44.],
          [ 64., -24.,  27.],
          [ 79., -24.,  37.],
          ...,
          [ 56., -25.,  37.],
          [ 51., -25.,  37.],
          [ 50., -20.,  42.]],
  
         [[ 52., -22.,  27.],
          [ 51., -21.,  27.],
          [ 54., -20.,  30.],
          ...,
          [ 43., -21.,  41.],
          [ 43., -20.,  40.],
          [ 42., -21.,  42.]],
  
         [[ 42., -17.,  42.],
          [ 39., -17.,  41.],
          [ 42., -17.,  39.],
          ...,
          [ 42., -22.,  42.],
          [ 42., -21.,  42.],
          [ 42., -21.,  42.]],
  
         ...,
  
         [[-43.,   6.,  47.],
          [-43.,   6.,  47.],
          [-43.,   6.,  47.],
          ...,
          [-43.,   6.,  47.],
          [-43.,   6.,  47.],
          [-43.,   6.,  47.]],
  
         [[-43.,   6.,  47.],
          [-43.,   6.,  47.],
          [-43.,   6.,  47.],
          ...,
          [-43.,   6.,  47.],
          [-43.,   6.,  47.],
       

In [14]:
print(data.keys())

dict_keys(['data', 'labels', 'id'])


In [15]:
print(data['data'].keys())

dict_keys(['ACC', 'BVP', 'EDA', 'TEMP'])


In [16]:
ACC_SF = 32
BVP_SF = 64
EDA_SF = 4
TEMP_SF = 4
LABELS_SF = 700  

acc_df = pd.DataFrame(data['data']['ACC'].reshape(-1, 3), columns=[f'ACC_{i}' for i in range(3)])
bvp_df = pd.DataFrame(data['data']['BVP'].reshape(-1, 1), columns=['BVP'])
eda_df = pd.DataFrame(data['data']['EDA'].reshape(-1, 1), columns=['EDA'])
temp_df = pd.DataFrame(data['data']['TEMP'].reshape(-1, 1), columns=['TEMP'])

acc_df.index = pd.date_range(start='2024-01-01', periods=len(acc_df), freq=f'{1/ACC_SF}S')
bvp_df.index = pd.date_range(start='2024-01-01', periods=len(bvp_df), freq=f'{1/BVP_SF}S')
eda_df.index = pd.date_range(start='2024-01-01', periods=len(eda_df), freq=f'{1/EDA_SF}S')
temp_df.index = pd.date_range(start='2024-01-01', periods=len(temp_df), freq=f'{1/TEMP_SF}S')

labels_index = pd.date_range(start='2024-01-01', periods=len(data['labels']), freq='700L')
labels_series = pd.Series(data['labels'], index=labels_index)

combined_df = pd.concat([acc_df, bvp_df, eda_df, temp_df], axis=1)
aligned_labels = labels_series.reindex(combined_df.index, method='nearest')

# pridávam id
combined_df['labels'] = aligned_labels
repeated_ids = np.repeat(data['id'], len(combined_df) // len(data['id']) + 1)[:len(combined_df)]
combined_df['id'] = repeated_ids

In [17]:
combined_df

,ACC_0,ACC_1,ACC_2,BVP,EDA,TEMP,labels,id
2024-01-01 00:00:00.000000,55.0,-32.0,44.0,30.52,1.634132,35.81,0.0,2
2024-01-01 00:00:00.015625,NaN,NaN,NaN,29.56,NaN,NaN,0.0,2
2024-01-01 00:00:00.031250,64.0,-24.0,27.0,29.31,NaN,NaN,0.0,2
2024-01-01 00:00:00.046875,NaN,NaN,NaN,29.18,NaN,NaN,0.0,2
2024-01-01 00:00:00.062500,79.0,-24.0,37.0,28.91,NaN,NaN,0.0,2
...,...,...,...,...,...,...,...,...
2024-01-02 00:58:35.921875,NaN,NaN,NaN,-1.09,NaN,NaN,3.0,17
2024-01-02 00:58:35.937500,-43.0,6.0,47.0,-2.26,NaN,NaN,3.0,17
2024-01-02 00:58:35.953125,NaN,NaN,NaN,-3.43,NaN,NaN,3.0,17
2024-01-02 00:58:35.968750,-43.0,6.0,47.0,-4.61,NaN,NaN,3.0,17


In [18]:
combined_df["labels"].value_counts()

labels
3.0    2784227
0.0    1577924
1.0     892954
2.0     499519
Name: count, dtype: int64

In [19]:
cleaned_df = combined_df.dropna()

In [20]:
cleaned_df

,ACC_0,ACC_1,ACC_2,BVP,EDA,TEMP,labels,id
2024-01-01 00:00:00.000,55.0,-32.0,44.0,30.52,1.634132,35.81,0.0,2
2024-01-01 00:00:00.250,49.0,-21.0,30.0,52.54,1.614912,35.81,0.0,2
2024-01-01 00:00:00.500,52.0,-22.0,27.0,94.31,1.591848,35.81,0.0,2
2024-01-01 00:00:00.750,58.0,-14.0,28.0,-38.86,1.558534,35.81,0.0,2
2024-01-01 00:00:01.000,52.0,-22.0,27.0,94.31,1.591848,35.81,0.0,2
...,...,...,...,...,...,...,...,...
2024-01-02 00:58:34.750,-43.0,6.0,47.0,-17.38,0.858816,32.23,3.0,17
2024-01-02 00:58:35.000,-43.0,6.0,47.0,5.51,0.858816,32.21,3.0,17
2024-01-02 00:58:35.250,-43.0,6.0,47.0,-17.38,0.858816,32.23,3.0,17
2024-01-02 00:58:35.500,-43.0,6.0,47.0,8.78,0.860096,32.23,3.0,17


In [21]:
cleaned_df["labels"].value_counts()

labels
3.0    174014
0.0     98621
1.0     55810
2.0     31219
Name: count, dtype: int64

In [22]:
combined_df.to_csv('WESAD_All_4.csv', index=False)

In [47]:
# KLontrola
label_1_df = cleaned_df[cleaned_df["labels"] == 0]
label_1_df

,ACC_0,ACC_1,ACC_2,BVP,EDA,TEMP,labels,id
2024-01-01 00:00:00.000,126.326243,-1012.375512,1296.163595,13.570802,5452.450365,10279.722091,0.0,2
2024-01-01 00:00:00.250,55.142882,-635.294579,753.130788,23.360416,5366.846848,10279.722091,0.0,2
2024-01-01 00:00:00.500,90.734562,-669.574663,636.766616,41.930454,5264.122627,10279.722091,0.0,2
2024-01-01 00:00:00.750,161.917923,-395.333985,675.554673,-17.274044,5115.746165,10279.722091,0.0,2
2024-01-01 00:00:01.000,90.734562,-669.574663,636.766616,41.930454,5264.122627,10279.722091,0.0,2
...,...,...,...,...,...,...,...,...
2024-01-01 21:52:44.750,57.652528,918.940551,887.251551,-21.989961,-1141.515991,-11412.917628,0.0,17
2024-01-01 21:52:45.000,57.652528,918.940551,887.251551,6.970496,-1141.515991,-11802.683112,0.0,17
2024-01-01 21:52:45.250,57.652528,918.940551,887.251551,-21.989961,-1141.515991,-11412.917628,0.0,17
2024-01-01 21:52:45.500,57.652528,918.940551,887.251551,11.107704,-1136.589079,-11412.917628,0.0,17


In [51]:
id_1_df = cleaned_df[cleaned_df["id"] == "3"]
id_1_df

,ACC_0,ACC_1,ACC_2,BVP,EDA,TEMP,labels,id
2024-01-01 01:25:33.000,-11.086215,-414.204053,383.042854,15.714244,4577.287725,6674.579321,0.0,3
2024-01-01 01:25:33.250,-11.086215,-414.204053,383.042854,-6.605105,4577.287725,6674.579321,0.0,3
2024-01-01 01:25:33.500,-11.086215,-414.204053,383.042854,-50.199724,4570.631500,6674.579321,1.0,3
2024-01-01 01:25:33.750,-11.086215,-414.204053,383.042854,14.550352,4563.976314,6674.579321,1.0,3
2024-01-01 01:25:34.000,-11.086215,-414.204053,383.042854,-50.199724,4570.631500,6674.579321,1.0,3
...,...,...,...,...,...,...,...,...
2024-01-01 02:52:11.750,113.825460,-424.070394,-444.376783,-62.430979,-4186.446215,7382.626174,0.0,3
2024-01-01 02:52:12.000,113.825460,-424.070394,-444.376783,-9.828335,-4175.871643,7382.626174,0.0,3
2024-01-01 02:52:12.250,113.825460,-424.070394,-444.376783,-62.430979,-4186.446215,7382.626174,0.0,3
2024-01-01 02:52:12.500,113.825460,-424.070394,-444.376783,37.146828,-4218.169931,7382.626174,0.0,3


In [53]:
cleaned_df.to_csv('WESAD_All_W.csv', index=False) #uloženie hodnôt kde som odrsránil prázdne premenné

In [63]:
# Kontrola prázdnych hodnôt
nan_df = cleaned_df[pd.isna(cleaned_df).any(axis=1)]
print(nan_df)

Empty DataFrame
Columns: [ACC_0, ACC_1, ACC_2, BVP, EDA, TEMP, labels, id]
Index: []


In [72]:
# Interpolácia chýbajúcich hodnôt 
interpolated_df = combined_df.interpolate(method='linear')

C:\Users\admin\AppData\Local\Temp\ipykernel_2888\1945346133.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  interpolated_df = combined_df.interpolate(method='linear')


In [73]:
interpolated_df

,ACC_0,ACC_1,ACC_2,BVP,EDA,TEMP,labels,id
2024-01-01 00:00:00.000000,126.326243,-1012.375512,1296.163595,13.570802,5452.450365,10279.722091,0.0,2
2024-01-01 00:00:00.015625,179.713763,-875.255173,966.465105,13.144006,5447.100145,10279.722091,0.0,2
2024-01-01 00:00:00.031250,233.101284,-738.134833,636.766616,13.032862,5441.749925,10279.722091,0.0,2
2024-01-01 00:00:00.046875,322.080484,-738.134833,830.706904,12.975067,5436.399706,10279.722091,0.0,2
2024-01-01 00:00:00.062500,411.059685,-738.134833,1024.647192,12.855030,5431.049486,10279.722091,0.0,2
...,...,...,...,...,...,...,...,...
2024-01-01 21:52:45.921875,57.652528,918.940551,887.251551,-1.379833,-1151.365964,-11412.917628,0.0,17
2024-01-01 21:52:45.937500,57.652528,918.940551,887.251551,-2.860118,-1151.365964,-11412.917628,0.0,17
2024-01-01 21:52:45.953125,57.652528,918.940551,887.251551,-4.340404,-1151.365964,-11412.917628,0.0,17
2024-01-01 21:52:45.968750,57.652528,918.940551,887.251551,-5.833341,-1151.365964,-11412.917628,0.0,17


In [74]:
interpolated_df.to_csv('WESAD_All_I.csv', index=False) #uloženie hodnôt kde som interpoloval prázdne premenné